# Data Cleansing and Combination
------------------------------

1. Extract the isotope data from the original CNIP data
2. Remove unnecessary columns and clean the data
3. Extract the needed information from the HydroGFD data
4. Combine CNIP and HydroGFD data into one dataframe
5. Remove rows with missing values and save the data

I will begin by importing the libraries I will need for this notebook.

In [1]:
import pandas as pd
import numpy as np
import netCDF4 as nc
import glob
from datetime import datetime, timedelta

## 1. Extract the isotope data from the original CNIP data

The original dataset is has each station separated by sheet which is not helpful. So In this section we will be taking that data in and extracting the isotope data from it. From here I will be combining the data from the different sheets into one dataframe. The columns that I will be extracting are the following:
* Station
* Lat
* Lon
* Alt
* Date
* Month
* O18avg
* H2avg
* Dex

In [2]:
# List the stations within the CNIP dataset to extract
stationList = ["BAB", "BON", "CPA", "EGB", "ELA", "EST", "GOB", "HAB", "SAT", "SNA", "SKT", "OTT", "BRA", "CAM", "EUR", "RES", "ALR", "HAL"]
cnipDict = {} # Dictionary to store the CNIP data
stationCoords = {} # Dict to store the station coordinates
# Loop through each station and extract the data
for station in stationList:
    # Read in the data
    data = pd.read_excel("CNIP Updated Data Stations 10.08.2009..xls", header=None, sheet_name=station, skiprows=[0, 1])

    #Making sure the columns are filled properly with constant values: Station, Lat, Long, Alt
    data[0] = station #Station
    data[1] = data[1].iloc[0] #Lat
    data[2] = data[2].iloc[0] #Long
    data[3] = data[3].iloc[0] #Alt
    stationCoords[station] = tuple([data[1].iloc[0], data[2].iloc[0]]) #Store the station coordinates
    #Add the station data to the dictionary
    cnipDict[station] = data

columns = ["Station", "Lat", "Long", "Alt", "Date", "Month", "O18(1)", "O18(2)", "O18Avg", "H2(1)", "H2(2)", "H2avg", "dex", "Temp", "Prec(1)", "Prec(2)", "Prec(3)"]
# Combine the CNIP data into one dataframe
cnip = pd.concat(cnipDict.values(), ignore_index=True)
cnip.columns = columns

# Print the CNIP dataframe
cnip

,Station,Lat,Long,Alt,Date,Month,O18(1),O18(2),O18Avg,H2(1),H2(2),H2avg,dex,Temp,Prec(1),Prec(2),Prec(3)
0,BAB,47.98,55.82,190.0,1997-02-02,2.0,NaN,NaN,-10.520,NaN,NaN,-50.700,33.460,-8.2,84.5,NaN,8.1
1,BAB,47.98,55.82,190.0,1997-03-02,3.0,NaN,NaN,-8.605,NaN,NaN,-48.040,20.800,-6.4,164.6,NaN,12.1
2,BAB,47.98,55.82,190.0,1997-04-02,4.0,NaN,NaN,-10.880,NaN,NaN,-27.485,59.555,1.7,95.4,NaN,9.0
3,BAB,47.98,55.82,190.0,1997-05-02,5.0,NaN,NaN,-7.550,NaN,NaN,-50.455,9.945,NaN,NaN,NaN,12.4
4,BAB,47.98,55.82,190.0,1997-06-02,6.0,NaN,NaN,-5.835,NaN,NaN,-76.480,-29.800,NaN,NaN,NaN,9.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3265,HAL,68.47,81.15,8.0,2007-03-02,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3266,HAL,68.47,81.15,8.0,2007-04-02,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3267,HAL,68.47,81.15,8.0,2007-05-02,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3268,HAL,68.47,81.15,8.0,2007-06-02,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2. Remove unnecessary columns and clean the data

Now that it is in one dataframe I will be cleaning the data by removing any rows that have missing values, and converting the date into a date time object. This will also require that I make sure rows filled with empty strings are converted to NaN values. I will also remove columns that are not needed for the analysis, which are as follows:
* Month
* O18(1)
* O18(2)
* H2(1)
* H2(2)
* Temp
* Precip(1-3)


In [3]:
# Convert the date column to datetime format
cnip["Date"] = pd.to_datetime(cnip["Date"], format="%Y-%m-%d")

# Filling in empty strings with NaN
cnip = cnip.replace(r'^\s*$', np.nan, regex=True)

# Removing unnecessary columns
cnip = cnip.drop(["Month", "O18(1)", "O18(2)", "H2(1)", "H2(2)", "Prec(1)", "Prec(2)", "Prec(3)", "Temp"], axis=1)

# Removing rows in the Date column with NaT values
cnip = cnip[cnip.Date.isnull() == False].reset_index()
cnip

,index,Station,Lat,Long,Alt,Date,O18Avg,H2avg,dex
0,0,BAB,47.98,55.82,190.0,1997-02-02,-10.520,-50.700,33.460
1,1,BAB,47.98,55.82,190.0,1997-03-02,-8.605,-48.040,20.800
2,2,BAB,47.98,55.82,190.0,1997-04-02,-10.880,-27.485,59.555
3,3,BAB,47.98,55.82,190.0,1997-05-02,-7.550,-50.455,9.945
4,4,BAB,47.98,55.82,190.0,1997-06-02,-5.835,-76.480,-29.800
...,...,...,...,...,...,...,...,...,...
3244,3265,HAL,68.47,81.15,8.0,2007-03-02,NaN,NaN,NaN
3245,3266,HAL,68.47,81.15,8.0,2007-04-02,NaN,NaN,NaN
3246,3267,HAL,68.47,81.15,8.0,2007-05-02,NaN,NaN,NaN
3247,3268,HAL,68.47,81.15,8.0,2007-06-02,NaN,NaN,NaN


## 3. Extract the needed information from the HydroGFD data
There is 14 netcdf files.... But yeah we will extract that information and then combine it into one dataframe. This will require both precipitation flux and Mean temperature 2m. Downloaded [here](https://cds.climate.copernicus.eu/cdsapp#!/dataset/sis-ecv-cmip5-bias-corrected?tab=form) with these parameters:
* Variable: Mean temperature 2m / precipitation flux
* Model: GFDL-CM3 (NOAA, USA)
* Experiment: RCP 4.5
* Period: 1960 - 2010

This section will be broken up into subparts

### 3.1 Extract the precipitation flux data
This will be done by extracting the data from the netcdf files and then combining them into one dataframe. The columns that will be extracted are as follows:
* Lat
* Lon
* Precipitation

It is worth noting that the latitude used from the latitude are the ones that are closest to the stations latitude. The same goes for the longitude. This is done to make sure that the data is as accurate as possible.

In [4]:
# All the precipitation file starts with a "prAdjust" and ends with a ".nc"
path = "HydroGFD/prAdjust*"
precipFiles = glob.glob(path) #Creates a list of all the precipitation flux files relative paths

# Loop through each precipitation file and extract the data which should only be separated by time
# and store them in a single dataframe
precip = pd.DataFrame(columns=["Station","Lat", "Long", "Time", "Precipitation"])

# Loop through each file and pull out the data at each time step for every lat and lon coordinate we have in the CNIP dataset that is stored in the stationCoords dictionary
for file in precipFiles:
    ncid = nc.Dataset(file, "r")

    #Pull out the time data and coordiante data
    time = ncid.variables["time"][:].filled(np.nan)
    lat = ncid.variables["lat"][:].filled(np.nan)
    lon = ncid.variables["lon"][:].filled(np.nan)

    for stat, coords in stationCoords.items():
        latIndex = (np.abs(lat - coords[0])).argmin()
        lonIndex = (np.abs(lon - coords[1])).argmin()

        # Pull out the precipitation data at each time step
        precipData = ncid.variables["prAdjust"][:, latIndex, lonIndex].filled(0) #Filling with 0 is an assumption that if there is no data, then there is no precipitation
        
        # Place the lat, lon, time, and precipitation data into a dataframe
        df = pd.DataFrame({"Station": stat, "Lat": coords[0], "Long": coords[1], "Time": time, "Precipitation": precipData})
        precip = pd.concat([precip, df], ignore_index=True)
    print("Finished extracting data from " + file[-20:-3])
    ncid.close()

# Convert the time data to datetime format
precip["Time"] = precip["Time"].apply(lambda x: datetime(1850, 1, 1) + timedelta(days=x))

C:\Users\jaxgr\AppData\Local\Temp\ipykernel_17592\3512970601.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  precip = pd.concat([precip, df], ignore_index=True)


Finished extracting data from 19600101-19641231
Finished extracting data from 19650101-19691231
Finished extracting data from 19700101-19741231
Finished extracting data from 19750101-19791231
Finished extracting data from 19800101-19841231
Finished extracting data from 19850101-19891231
Finished extracting data from 19900101-19941231
Finished extracting data from 19950101-19991231
Finished extracting data from 20000101-20041231
Finished extracting data from 20050101-20051231
Finished extracting data from 20060101-20101231


In [5]:
#Sort the dataframe by time and reset the index
precip = precip.sort_values(by=["Time"]).reset_index()

### 3.2 Extract the mean temperature data
This will be done almost exactly the same way as it was done for the precipitation flux data. The columns that will be extracted are as follows:
* Lat
* Lon
* Temperature

It is worth noting that the latitude used from the latitude are the ones that are closest to the stations latitude. The same goes for the longitude. This is done to make sure that the data is as accurate as possible.

In [6]:
# All the temperature files starts with a "tasAdjust" and ends with a ".nc"
path = "HydroGFD/tasAdjust*"
tempFiles = glob.glob(path) #Creates a list of all the temperature flux files relative paths

# Loop through each temperature file and extract the data which should only be separated by time
# and store them in a single dataframe
temperature = pd.DataFrame(columns=["Station","Lat", "Long", "Time", "Temperature"])

# Loop through each file and pull out the data at each time step for every lat and lon coordinate we have in the CNIP dataset that is stored in the stationCoords dictionary
for file in tempFiles:
    ncid = nc.Dataset(file, "r")

    #Pull out the time data and coordiante data
    time = ncid.variables["time"][:].filled(np.nan)
    lat = ncid.variables["lat"][:].filled(np.nan)
    lon = ncid.variables["lon"][:].filled(np.nan)

    for stat, coords in stationCoords.items():
        latIndex = (np.abs(lat - coords[0])).argmin()
        lonIndex = (np.abs(lon - coords[1])).argmin()

        # Pull out the temperature data at each time step
        tempData = ncid.variables["tasAdjust"][:, latIndex, lonIndex].filled(np.nan) #Filling with nan, as we can't make an assumption about the temperature
        
        # Place the lat, lon, time, and temperature data into a dataframe
        df = pd.DataFrame({"Station": stat, "Lat": coords[0], "Long": coords[1], "Time": time, "Temperature": tempData})
        temperature = pd.concat([temperature, df], ignore_index=True)
    print("Finished extracting data from " + file[-20:-3])
    ncid.close()

# Convert the time data to datetime format
temperature["Time"] = temperature["Time"].apply(lambda x: datetime(1850, 1, 1) + timedelta(days=x))

C:\Users\jaxgr\AppData\Local\Temp\ipykernel_17592\2826800507.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  temperature = pd.concat([temperature, df], ignore_index=True)


Finished extracting data from 19600101-19641231
Finished extracting data from 19650101-19691231
Finished extracting data from 19700101-19741231
Finished extracting data from 19750101-19791231
Finished extracting data from 19800101-19841231
Finished extracting data from 19850101-19891231
Finished extracting data from 19900101-19941231
Finished extracting data from 19950101-19991231
Finished extracting data from 20000101-20041231
Finished extracting data from 20050101-20051231
Finished extracting data from 20060101-20101231


In [7]:
#Sort the dataframe by time and reset the index
temperature = temperature.sort_values(by=["Time"]).reset_index()

### 3.3 Combine the precipitation flux and mean temperature data
Now that we have the precipitation flux and mean temperature data we will combine them into one dataframe. This will require that we merge the two dataframes on latitude, longitude, station, and time. This will be done using the pandas merge function. The columns that will be in the final dataframe are as follows:
* Station
* Lat
* Lon
* Time
* Precipitation
* Temperature

In [8]:
# Merge the precipitation and temperature dataframes
hydroGFD = pd.merge(precip, temperature, on=["Station", "Lat", "Long", "Time"], how="outer")

# Keep only the needed columns of the dataframe: station, lat, long, time, precipitation, temperature
# removing the index columns
hydroGFD = hydroGFD[["Station", "Lat", "Long", "Time", "Precipitation", "Temperature"]]

# Renaming some columns to include units
hydroGFD = hydroGFD.rename(columns={
    "Precipitation": "Precipitation (kg/m^2/s)", 
    "Temperature": "Temperature (K)",
    "Time": "Date"})

# Finally saving this data as a CSV file just in case
hydroGFD.to_csv(r"hydroGFD.csv", index=False)

## 4. Combine CNIP and HydroGFD data into one dataframe
Now that we have the CNIP data and the HydroGFD data we will combine them into one dataframe. This will require that we merge the two dataframes on latitude, longitude, station, and time. This will be done using the pandas merge function. The columns that will be in the final dataframe are as follows:
* Station
* Lat
* Lon
* Alt
* Time / Date
* Precipitation (kg/m^2/s)
* Temperature (K)
* O18avg (‰)
* H2avg (‰)
* Dex (‰)


In [10]:
# I need to make sure that CNIP and HydroGFD have the same date format so that I can merge them
# I will also make sure that the date range is the same for both datasets
# Convert the date column to datetime format
cnip["Date"] = pd.to_datetime(cnip["Date"]).dt.date
hydroGFD["Date"] = pd.to_datetime(hydroGFD["Date"]).dt.date

# Remove the rows in the CNIP dataset that are not in the HydroGFD dataset and vice versa
cnip = cnip[cnip.Date.isin(hydroGFD.Date)].reset_index()
hydroGFD = hydroGFD[hydroGFD.Date.isin(cnip.Date)].reset_index()

ValueError: cannot insert level_0, already exists

In [15]:
# Merge the CNIP and HydroGFD datasets
combined = pd.merge(cnip, hydroGFD, on=["Station", "Lat", "Long", "Date"], how="outer")

# Sort the dataframe by date and reset the index
combined = combined.sort_values(by=["Date"]).reset_index()

# Remove the index column
combined = combined.drop(["index", "index_x", "index_y", "level_0"], axis=1)

## 5. Remove rows with missing values and save the dataframe as a csv file
Now that we have the combined dataframe we will remove any rows that have missing values. This will be done using the pandas dropna function.

In [21]:
# remove the rows with NaN values
dataset = combined.dropna().reset_index()

# Remove the index column
dataset = dataset.drop(["index"], axis=1)

# Make sure that the numeric columns are in the float64 format
dataset["O18Avg"] = dataset["O18Avg"].astype("float64")
dataset["H2avg"] = dataset["H2avg"].astype("float64")
dataset["dex"] = dataset["dex"].astype("float64")
dataset["Precipitation (kg/m^2/s)"] = dataset["Precipitation (kg/m^2/s)"].astype("float64")
dataset["Temperature (K)"] = dataset["Temperature (K)"].astype("float64")
dataset["Alt"] = dataset["Alt"].astype("float64")
dataset["Lat"] = dataset["Lat"].astype("float64")
dataset["Long"] = dataset["Long"].astype("float64")

# Finally saving this data as a CSV file
dataset.to_csv(r"Isoscape_Data.csv", index=False)